In [2]:
import pandas as pd
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import networkx as nx
import os  
import ast #to convert the string into dict
from datetime import datetime as dt
import requests
import dask.dataframe as dd


In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [10]:
df_articles = pd.read_csv('./Articles/newsarticles.csv')

In [4]:
df_articles['body_no_html'] = df_articles['body'].apply(remove_html_tags)

In [11]:
#delete rows without user needs (265)
df_articles.dropna(subset=['userNeeds'],inplace = True)
df_articles.reset_index(drop=True,inplace=True)

In [12]:
#the dictionary is reversed --> now we have principal/secondaire as keys
dict_column = []
for i in range(0,df_articles.shape[0]):
    inv_map = dict(zip(ast.literal_eval(df_articles['userNeeds'][i]).values(), ast.literal_eval(df_articles['userNeeds'][i]).keys()))
    dict_column.append(inv_map)

df_articles['userNeeds_dict'] = dict_column
userNeed = []
for i in range(0,df_articles.shape[0]):
    userNeed.append(df_articles['userNeeds_dict'][i].get('principal'))

df_articles['userNeeds_clean'] = userNeed
df_articles.drop(['userNeeds_dict'],axis = 1,inplace=True)

In [14]:
df_articles.to_csv('Articles_withCleanedUN.csv')

## Topic Association

In [4]:
df_rts_0 = dd.read_csv('/www.rts.ch_accesslogs.log',sep = ' ', header = None,)
df_rts = df_rts_0.compute()

In [5]:
df_rts = df_rts.drop([0,1,3,5,6,7,8,10,11,12,13,14,16,17,18],axis = 1)
df_rts.columns = ['Timestamp','User_IP','Url','User_Agent']
index_html = df_rts.query('Url.str.contains(".html$", na= False)').index
print('Html number')
print(len(index_html))
df_rts_html = df_rts[df_rts.index.isin(index_html)]
df_rts_html.reset_index(inplace=True, drop=True)

Html number
2833789


In [33]:
df_rts_html.loc[65,'Url']

'info/culture/cinema/13891031-drii-winter-cascadeuses-et-la-ligne-recompenses-aux-prix-du-cinema-suisse.html'

In [6]:
topics = {'culture': r'culture',
          'economie': r'economie',
          'suisse': r'suisse',
          'monde': r'monde',
          'sciences-tech': r'sciences-tech',
          'sport': r'sport',
          'environnement' : r'environnement'}

def extract_topic_and_id(url):
    if pd.isna(url):
        return 'other', None
    for topic, pattern in topics.items():
        if re.search(pattern, url):
            escenic_id_match = re.search(r'/(\d+)-', url)
            escenic_id = escenic_id_match.group(1) if escenic_id_match else None
            return topic, escenic_id
    return 'other', None  # If no topic matches, return 'other' and None for Escenic ID

In [ ]:
# Apply the function to extract the topic and Escenic ID
df_rts_html[['Topic', 'Escenic_ID']] = df_rts_html['Url'].apply(lambda x: pd.Series(extract_topic_and_id(x)))

In [8]:
topic, escenic_id = extract_topic_and_id(df_rts_html.loc[65,'Url'])

print("Topic:", topic)
print("Escenic ID:", escenic_id)

Topic: culture
Escenic ID: 13891031
